In [2]:
import pandas as pd
import numpy as np
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import scrapy


In [3]:
conn = mysql.connector.connect(
    host="localhost",
    user="Himanshu",
    password="Hustling@2000",
    database="movies_db"
)
cursor = conn.cursor()

In [3]:
# SQL query to count null values for each column in movies_data table
sql_query = """
SELECT 
    COUNT(*) as total_rows,
    SUM(CASE WHEN title IS NULL THEN 1 ELSE 0 END) as title_nulls,
    SUM(CASE WHEN type IS NULL THEN 1 ELSE 0 END) as type_nulls,
    SUM(CASE WHEN director IS NULL THEN 1 ELSE 0 END) as director_nulls,
    SUM(CASE WHEN cast IS NULL THEN 1 ELSE 0 END) as cast_nulls,
    SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) as country_nulls,
    SUM(CASE WHEN date_added IS NULL THEN 1 ELSE 0 END) as date_added_nulls,
    SUM(CASE WHEN release_year IS NULL THEN 1 ELSE 0 END) as release_year_nulls,
    SUM(CASE WHEN rating IS NULL THEN 1 ELSE 0 END) as rating_nulls,
    SUM(CASE WHEN duration IS NULL THEN 1 ELSE 0 END) as duration_nulls,
    SUM(CASE WHEN listed_in IS NULL THEN 1 ELSE 0 END) as listed_in_nulls,
    SUM(CASE WHEN description IS NULL THEN 1 ELSE 0 END) as description_nulls
FROM movies_data;
"""

# Execute the query using your existing cursor
cursor.execute(sql_query)
results = cursor.fetchall()

# Convert results to a more readable format
column_names = [desc[0] for desc in cursor.description]
missing_values = dict(zip(column_names, results[0]))

# Print the results in a formatted way
print("\nMissing Values Analysis:")
print("-" * 50)
print(f"Total Rows: {missing_values['total_rows']}")
print("-" * 50)
for column, nulls in missing_values.items():
    if column != 'total_rows':
        percentage = (nulls / missing_values['total_rows']) * 100
        print(f"{column.replace('_nulls', '')}: {nulls} nulls ({percentage:.2f}%)")


Missing Values Analysis:
--------------------------------------------------
Total Rows: 3071
--------------------------------------------------
title: 0 nulls (0.00%)
type: 0 nulls (0.00%)
director: 3068 nulls (99.90%)
cast: 3071 nulls (100.00%)
country: 1452 nulls (47.28%)
date_added: 28 nulls (0.91%)
release_year: 0 nulls (0.00%)
rating: 519 nulls (16.90%)
duration: 478 nulls (15.56%)
listed_in: 0 nulls (0.00%)
description: 4 nulls (0.13%)


As we can see that there are many null values in the columns director, cast, country, date_added, rating and durations.
Most of them can be derived from the internet.
So now what I will be doing is web scrapping using BeautifulSoup.

In [ ]:
# SQL query to get 10 rows with NULL values in specified columns
sql_query = """
SELECT *
FROM movies_data
WHERE director IS NULL 
   OR cast IS NULL 
   OR country IS NULL 
   OR date_added IS NULL 
   OR rating IS NULL 
   OR duration IS NULL
LIMIT 10;
"""

# You can execute this query using your existing MySQL connection:
cursor.execute(sql_query)
results = cursor.fetchall()

# Convert results to a pandas DataFrame for better visualization
columns = [desc[0] for desc in cursor.description]
df_null_analysis = pd.DataFrame(results, columns=columns)

# Display the results
display(df_null_analysis)

In [ ]:
import pandas as pd

# SQL query to fetch rows where 'type' is 'Movie' and any of the specified columns are NULL
query = """
    SELECT * FROM movies_data 
    WHERE type = 'Movie' 
    AND (director IS NULL OR cast IS NULL OR country IS NULL OR date_added IS NULL OR rating IS NULL)
"""

# Execute query
cursor.execute(query)

# Fetch all results
missing_data_rows = cursor.fetchall()

# Get column names from the table
column_names = [desc[0] for desc in cursor.description]

# Convert to DataFrame
df_missing_values_movies = pd.DataFrame(missing_data_rows, columns=column_names)

# Display the DataFrame
display(df_missing_values_movies)


In [23]:
# Set up WebDriver with options
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Open IMDb website
    driver.get("https://www.imdb.com/")

    # Locate search bar and type the movie name
    movie_title = "City of God (2002)"  # Change this for any other movie
    print(f"\n[INFO] Searching for '{movie_title}' on IMDb...")

    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#suggestion-search"))
    )
    search_box.send_keys(movie_title)
    search_box.send_keys(Keys.RETURN)

    # Wait for search results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ul.ipc-metadata-list li a"))
    )

    # Click the first result dynamically
    first_result = driver.find_element(By.CSS_SELECTOR, "ul.ipc-metadata-list li a")
    first_result.click()

    # Wait for the movie page to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li[data-testid='title-pc-principal-credit']"))
    )

    # Extract all directors (handles both single & multiple directors)
    director_elements = driver.find_elements(By.CSS_SELECTOR, "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-1aa68851-0.gncgHj > section > div:nth-child(5) > section > section > div.sc-9a2a0028-4.eeUUGv > div.sc-9a2a0028-6.zHrZh > div.sc-9a2a0028-10.iUfJXd > section > div.sc-70a366cc-3.iwmAOx > div > ul > li:nth-child(1) > div > ul > li a")
    
    # Extract text and clean up data
    directors = [element.text.strip() for element in director_elements] 
    print(directors)

    # if directors:
    #     print(f"[INFO] Director(s) of '{movie_title}': ", end="")
    #     for i, director in enumerate(directors):
    #         if i < len(directors) - 1:
    #             print(f"{director}, ", end="")
    #         else:
    #             print(director)
    # else:
    #     print(f"[WARNING] No director found for '{movie_title}'.")

except Exception as e:
    print(f"[ERROR] Unexpected issue: {str(e)}")

finally:
    # Close the browser
    driver.quit()


[INFO] Searching for 'City of God (2002)' on IMDb...
['Fernando Meirelles', 'Kátia Lund']


In [21]:
import requests
from bs4 import BeautifulSoup
import time

def get_movie_directors(movie_title):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
    }

    # Step 1: Construct the IMDb search URL
    search_url = f"https://www.imdb.com/find/?q={movie_title.replace(' ', '+')}&s=tt"
    print(f"[INFO] Searching for '{movie_title}' on IMDb...")
    
    time.sleep(2)  # Simulating delay

    # Step 2: Fetch search results page
    search_response = requests.get(search_url, headers=headers)
    search_soup = BeautifulSoup(search_response.text, "html.parser")

    # Step 3: Find the div containing the search results
    results_div = search_soup.select_one("div.sc-b03627f1-2.gWHDBT > ul")
    if not results_div:
        print("[WARNING] No search results found!")
        return

    # Step 4: Iterate over each result to find the exact match
    movie_url = None
    for result in results_div.find_all("li"):
        movie_tag = result.select_one("div.ipc-metadata-list-summary-item__c div a")
        if movie_tag:
            movie_name = movie_tag.text.strip()
            if movie_name.lower() == movie_title.lower():  # Case insensitive comparison
                movie_url = "https://www.imdb.com" + movie_tag["href"]
                break

    if not movie_url:
        print(f"[WARNING] No exact match found for '{movie_title}'.")
        return

    print(f"[INFO] Found exact match: {movie_url}")

    time.sleep(2)  # Simulating delay

    # Step 5: Open the movie page and extract director(s)
    movie_response = requests.get(movie_url, headers=headers)
    movie_soup = BeautifulSoup(movie_response.text, "html.parser")

    director_elements = movie_soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-1aa68851-0.gncgHj > section > div:nth-child(5) > section > section > div.sc-9a2a0028-4.eeUUGv > div.sc-9a2a0028-6.zHrZh > div.sc-9a2a0028-10.iUfJXd > section > div.sc-70a366cc-3.iwmAOx > div > ul > li:nth-child(1) > div > ul > li a")
    directors = [director.text.strip() for director in director_elements]

    if directors:
        print(f"[INFO] Director(s) of '{movie_title}': {', '.join(directors)}")
    else:
        print(f"[WARNING] No director found for '{movie_title}'.")

# Run the function
get_movie_directors("Dhoom")


[INFO] Searching for 'Dhoom' on IMDb...
[INFO] Found exact match: https://www.imdb.com/title/tt0422091/?ref_=fn_ttl_ttl_2
[INFO] Director(s) of 'Dhoom': Sanjay Gadhvi


In [13]:
# Function to clean the title
def clean_title(title):
    return title.replace('(Dub)', '').replace('(Sub)', '').strip()

# Fetch titles with '(Dub)' or '(Sub)'
cursor.execute("SELECT show_id, title FROM movies_data WHERE title LIKE '%(Dub)%' OR title LIKE '%(Sub)%'")
rows = cursor.fetchall()

# Loop through each row
for show_id, old_title in rows:
    new_title = clean_title(old_title)

    # Check if cleaned title already exists
    cursor.execute("SELECT show_id FROM movies_data WHERE title = %s", (new_title,))
    existing_row = cursor.fetchone()

    if existing_row:
        # If duplicate exists, delete the current row
        delete_query = "DELETE FROM movies_data WHERE show_id = %s"
        cursor.execute(delete_query, (show_id,))
        conn.commit()
        print(f"[DELETED] Duplicate '{old_title}' (ID: {show_id}) since '{new_title}' already exists.")
    else:
        # If no duplicate, update the title
        update_query = "UPDATE movies_data SET title = %s WHERE show_id = %s"
        cursor.execute(update_query, (new_title, show_id))
        conn.commit()
        print(f"[UPDATED] {old_title} → {new_title}")

In [9]:
# SQL query to fetch top 10 rows where type is 'Movie' and at least one column is NULL
query = """
SELECT * 
FROM movies_data
WHERE type = 'Movie' 
AND (
    director IS NULL OR
    cast IS NULL OR
    country IS NULL OR
    date_added IS NULL OR
    release_year IS NULL OR
    rating IS NULL OR
    duration IS NULL OR
    listed_in IS NULL OR
    description IS NULL
);
"""

# Execute the query and fetch data
cursor.execute(query)
rows = cursor.fetchall()

# Fetch column names
columns = [desc[0] for desc in cursor.description]

# Create a DataFrame from fetched data
missing_movie_data = pd.DataFrame(rows, columns=columns)


# Display DataFrame
display(missing_movie_data)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1462,Movie,'71,None,None,United Kingdom,2020-08-01,2014,R,99 min,"Action, Adventure, Drama",Jack O'Connell is a British solider accidental...
1,s874,Movie,1 Night in San Diego,None,None,None,2021-03-15,2020,None,86 min,Comedy,"BFFs Hannah and Brooklyn, played by Jenna Ushk..."
2,s2414,Movie,10.0 Earthquake,None,None,United States,2018-09-28,2014,87 min,None,"Action, Adventure",Los Angeles is about to be hit by a devastatin...
3,s1723,Movie,100 Streets,None,None,United Kingdom,2020-04-15,2016,94 min,None,Drama,"Three people, three extraordinary stories. All..."
4,s860,Movie,100% Wolf,None,None,None,2021-03-23,2020,TV-PG,96 min,Comedy,"Freddy Lupin can't wait to become a werewolf, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1476,s1588,Movie,Your Home Made Perfect,None,None,United Kingdom,2020-06-15,2019,2 Seasons,None,Reality,Never before have plans come to life like this...
1477,s847,Movie,Zappa,None,None,None,2021-04-01,2020,None,128 min,"Documentaries, Music",ZAPPA is an expansive and intimate portrait of...
1478,s2416,Movie,Zapped,None,None,"Canada, United States",2018-09-28,2014,TV-G,94 min,"Adventure, Black Stories, Comedy","Zoey, a skilled dancer and straight-A student,..."
1479,s2019,Movie,ZOMBOAT!,None,None,United Kingdom,2019-10-25,2019,1 Season,None,"Comedy, Drama, International",Sisters Kat and Jo realise there's a zombie ap...


In [10]:
# Initialize WebDriver with options
options = Options()
options.add_argument("--headless")  # Run browser in background
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

# Create WebDriver instance
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Function to search movie on IMDb and get missing data
def get_movie_details(title, missing_columns):
    print(f"\n[INFO] Searching for '{title}' on IMDb...")
    driver.get("https://www.imdb.com/")
    
    # Wait and find search bar
    search_box = driver.find_element(By.CSS_SELECTOR, "#suggestion-search")
    search_box.send_keys(title)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Wait for results to load
    
    # Click on the first relevant result
    # try:
    #     first_result = driver.find_element(By.CSS_SELECTOR, "td.result_text a")
    #     first_result.click()
    #     time.sleep(3)  # Wait for movie page to load
    # except:
    #     print(f"[ERROR] Could not find movie '{title}' on IMDb.")
    #     return {}
    
    try:
    # Wait until the first search result is clickable
        first_result = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-54536c3d-0.kFmgEJ > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a"))
        )  # Wait for movie page to load
        driver.execute_script("arguments[0].click();", first_result)
        time.sleep(5)  # Allow time for movie page to load  # Wait for the movie page to load
    except NoSuchElementException:
        print(f"[ERROR] No search results for '{title}'.")
        return {}

    except TimeoutException:
            print("[ERROR] IMDb search page took too long to load.")
            return {}

    except Exception as e:
        print(f"[ERROR] Unexpected issue: {str(e)}")
        return {}

    movie_data = {}
    # Fetch missing details
    if "director" in missing_columns:
        try:
            # Find all director elements (targeting individual <a> tags inside the <ul>)
            director_elements = driver.find_elements(By.CSS_SELECTOR, "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-1aa68851-0.gncgHj > section > div:nth-child(5) > section > section > div.sc-9a2a0028-4.eeUUGv > div.sc-9a2a0028-6.zHrZh > div.sc-9a2a0028-10.iUfJXd > section > div.sc-70a366cc-3.iwmAOx > div > ul > li:nth-child(1) > div > ul > li a")
            
            # Extract text from each director element and clean it
            directors = [element.text.strip() for element in director_elements]

            if directors:
                movie_data["director"] = ", ".join(directors)  # Store as a comma-separated string
                print(f"  [INFO] Found Director(s): {movie_data['director']}")
            else:
                print(f"  [WARNING] No directors found for '{title}'.")

        except Exception as e:
            print(f"  [ERROR] Could not fetch director for '{title}': {str(e)}")


    if "country" in missing_columns:
        try:
            country = driver.find_element(By.XPATH, "//li[@data-testid='title-details-origin']//a").text
            movie_data["country"] = country
            print(f"  [INFO] Found Country: {country}")
        except:
            print(f"  [WARNING] Country not found for '{title}'.")

    if "rating" in missing_columns:
        try:
            rating = driver.find_element(By.CSS_SELECTOR, "span.sc-1fb98db9-1.gTzKIg").text
            movie_data["rating"] = rating
            print(f"  [INFO] Found Rating: {rating}")
        except:
            print(f"  [WARNING] Rating not found for '{title}'.")

    if "date_added" in missing_columns:
        try:
            date_added = driver.find_element(By.XPATH, "//li[@data-testid='title-details-release-date']//a").text
            movie_data["date_addeds"] = date_added
            print(f"  [INFO] Found Release Date: {date_added}")
        except:
            print(f"  [WARNING] Release Date not found for '{title}'.")

    if "cast" in missing_columns:
        try:
            cast = driver.find_element(By.XPATH, "//a[@data-testid='title-cast-item__actor']").text
            movie_data["cast"] = cast
            print(f"  [INFO] Found Lead Actor: {cast}")
        except:
            print(f"  [WARNING] Cast not found for '{title}'.")

    return movie_data

# Iterate through missing_movie_data DataFrame
for index, row in missing_movie_data.iterrows():
    title = row["title"]
    missing_columns = [col for col in missing_movie_data.columns if pd.isnull(row[col])]

    if not missing_columns:
        print(f"\n[INFO] No missing data for '{title}', skipping...")
        continue

    print(f"\n[INFO] Processing movie: {title}")
    print(f"  [INFO] Missing Columns: {missing_columns}")

    # Fetch missing data from IMDb
    movie_details = get_movie_details(title, missing_columns)

    # Update DataFrame with fetched values
    for col, value in movie_details.items():
        missing_movie_data.at[index, col] = value

    print(f"  [SUCCESS] Updated '{title}' with new details.")

# Close WebDriver
driver.quit()

# Display updated DataFrame
print("\n[INFO] Updated DataFrame:")
print(missing_movie_data)


[INFO] Processing movie: '71
  [INFO] Missing Columns: ['director', 'cast']

[INFO] Searching for ''71' on IMDb...
  [INFO] Found Director(s): Yann Demange
  [INFO] Found Lead Actor: Jack O'Connell
  [SUCCESS] Updated ''71' with new details.

[INFO] Processing movie: 1 Night in San Diego
  [INFO] Missing Columns: ['director', 'cast', 'country', 'rating']

[INFO] Searching for '1 Night in San Diego' on IMDb...
  [INFO] Found Director(s): Penelope Lawson
  [INFO] Found Country: United States
  [WARNING] Rating not found for '1 Night in San Diego'.
  [INFO] Found Lead Actor: Jenna Ushkowitz
  [SUCCESS] Updated '1 Night in San Diego' with new details.

[INFO] Processing movie: 10.0 Earthquake
  [INFO] Missing Columns: ['director', 'cast', 'duration']

[INFO] Searching for '10.0 Earthquake' on IMDb...
  [INFO] Found Director(s): David Gidali
  [INFO] Found Lead Actor: Jeffrey Jones
  [SUCCESS] Updated '10.0 Earthquake' with new details.

[INFO] Processing movie: 100 Streets
  [INFO] Missin

KeyboardInterrupt: 

In [11]:
display(missing_movie_data)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1462,Movie,'71,Yann Demange,Jack O'Connell,United Kingdom,2020-08-01,2014,R,99 min,"Action, Adventure, Drama",Jack O'Connell is a British solider accidental...
1,s874,Movie,1 Night in San Diego,Penelope Lawson,Jenna Ushkowitz,United States,2021-03-15,2020,None,86 min,Comedy,"BFFs Hannah and Brooklyn, played by Jenna Ushk..."
2,s2414,Movie,10.0 Earthquake,David Gidali,Jeffrey Jones,United States,2018-09-28,2014,87 min,None,"Action, Adventure",Los Angeles is about to be hit by a devastatin...
3,s1723,Movie,100 Streets,Jim O'Hanlon,Idris Elba,United Kingdom,2020-04-15,2016,94 min,None,Drama,"Three people, three extraordinary stories. All..."
4,s860,Movie,100% Wolf,Alexs Stadermann,Loren Gray,Australia,2021-03-23,2020,TV-PG,96 min,Comedy,"Freddy Lupin can't wait to become a werewolf, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1476,s1588,Movie,Your Home Made Perfect,None,None,United Kingdom,2020-06-15,2019,2 Seasons,None,Reality,Never before have plans come to life like this...
1477,s847,Movie,Zappa,None,None,None,2021-04-01,2020,None,128 min,"Documentaries, Music",ZAPPA is an expansive and intimate portrait of...
1478,s2416,Movie,Zapped,None,None,"Canada, United States",2018-09-28,2014,TV-G,94 min,"Adventure, Black Stories, Comedy","Zoey, a skilled dancer and straight-A student,..."
1479,s2019,Movie,ZOMBOAT!,None,None,United Kingdom,2019-10-25,2019,1 Season,None,"Comedy, Drama, International",Sisters Kat and Jo realise there's a zombie ap...


We can see that we have Sub and Dub written for the movies name. Probably it will be there for the TV shows name. Now will be editing it in the dtabase directly. That would be better.